In [1]:
import numpy as np
import sys
sys.path.append('.')
from step13_main import *

## 13. 가변 길이 인수 (역전파)
### 13.1. 가변 길이 인수에 대응한 Add 클래스의 역전파
덧셈에 대한 역전파는 gy를 그대로 위로 흘려 보내주면 된다는 컨셉을 이용해서 진행하면 된다.
```python
class Add(Function):
    ...
    def backward(self, gy):
        return gy, gy
```
### 13.2. Variable 클래스 수정
역전파에 대한 가변인수를 처리하기 위한 수정을 진행하면 된다!
 = *인자 형식으로 언팩 후 리스트 이용하기
```python
class Variable:
    ...
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] # 각 인수에서 grad(기울기) 추출
            gxs = f.backward(*gys) # 언팩
            if not isinstance(gxs, tuple): # 튜플 변환
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # 각 역전파 결과값(다음 차례의 gy)에 대해서
                x.grad = gx # gy 설정해주기

                if x.creator is not None: # 초기 단계의 Var가 아닌 이상
                    funcs.append(x.creator) # 다음 차례의 함수 붙여서 역전파 계속하기
```
### 13.3. Square 클래스 구현
역전파에서 가변인수로 받으니 그 중에서 인덱스 0번을 가져오는 형태
```python
class Square(Function):
    ...
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
```

In [ ]:
x = Variable(np.array(2))
y = Variable(np.array(3))

z = add(square(x), square(y))

In [3]:
z.backward()
print(z.data)
print(x.grad)
print(y.grad)





13
4
6
